In [5]:
import os
import sys
import urllib.request
import json
client_id = "aY4Gxe5F9WdX0P2JxiBz" # 발급받은 id 입력
client_secret = "DMple3S42_" # 발급받은 secret 입력 
quote = input("검색어를 입력해주세요.: ") #검색어 입력받기
encText = urllib.parse.quote(quote)
display_num = input("검색 출력결과 갯수를 적어주세요.(최대100, 숫자만 입력): ") #출력할 갯수 입력받기
url = "https://openapi.naver.com/v1/search/blog?query=" + encText +"&display="+display_num# json 결과
# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id", "aY4Gxe5F9WdX0P2JxiBz")
request.add_header("X-Naver-Client-Secret", "DMple3S42_")
response = urllib.request.urlopen(request)
rescode = response.getcode()

if(rescode==200):
    response_body = response.read()
    #print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

body = response_body.decode('utf-8')
body

# 불필요한 ""(큰따옴표)지워주기
body = body.replace('"','')
#print(body)

#블로그 제목만 뽑기
import re
titles = re.findall('title: (.*?),\nlink', body) #\n은 엔터의 의미
print('<<제목 모음>>')
print(titles)
print('총 제목 수: ',len(titles),'개')#제목갯수확인

#블로그 링크들 추출
links = re.findall('link: (.*?),\ndescription',body)
print('\n<<링크 모음>>')
print(links)
print('총 링크 수: ',len(links),'개')#링크갯수확인

# 제목 다듬기
blog_titles = []
for k in titles:
    a = k.replace('<b>','')
    b = a.replace('</b>', '')
    blog_titles.append(b)

# 링크를 다듬기 (필요없는 부분 제거 및 수정)
blog_links = []
for i in links:
    a = i.replace('\\','')
    b = a.replace('?Redirect=Log&logNo=','/')
    blog_links.append(b)

print(blog_links)
print('생성된 링크 갯수:',len(blog_links),'개')

#본문 크롤링
import time
from selenium import webdriver
from bs4 import BeautifulSoup

driver = webdriver.Chrome(os.path.abspath('chromedriver')) #또는 driver.exe 가있는 파일 위치 복사하여 입력
driver.implicitly_wait(3)

#블로그 링크 하나씩 불러서 크롤링
contents = []
for i in blog_links:
    #블로그 링크 하나씩 불러오기
    driver.get(i)
    time.sleep(1)
    #블로그 안 본문이 있는 iframe에 접근하기
    driver.switch_to.frame("mainFrame")
    #본문 내용 크롤링하기
    try:
        a = driver.find_element_by_css_selector('div.se-main-container').text
        contents.append(a)
    # NoSuchElement 오류시 예외처리(구버전 블로그에 적용)
    except:
        a = driver.find_element_by_css_selector('div#content-area').text
        contents.append(a)
    #print(본문: \n', a)




driver.quit() #창닫기
print("<<본문 크롤링이 완료되었습니다.>>")

#제목 및 본문 txt에 저장
total_contents = titles + contents

text = open("blog_text.txt",'w',encoding='utf-8') 
for i in total_contents:
    text.write(i)
text.close()

#제목, 블로그링크, 본문내용 Dataframe으로 만들기
import pandas as pd

df = pd.DataFrame({'제목':blog_titles, '링크':blog_links,'내용':contents})
df

<<제목 모음>>
['<b>무신사</b> 크림 <b>가품 논란</b>, 에센셜 티셔츠 정품 인증에 대한 짝퉁... ', '<b>무신사</b> 크림 피어오브갓 에센셜티셔츠 <b>가품 논란</b> 결과는?', '<b>무신사</b> - 크림 피어오브갓 에센셜 티셔츠 <b>가품 논란</b>', '골프의 계절, MZ세대를 노려라!+<b>무신사</b>VS크림, <b>가품논란</b>... ', '[유통이슈] <b>무신사</b>, 크림 <b>가품 논란</b>에 대해 알아봅시다!', '<b>무신사</b> 크림 <b>가품 논란</b> 총정리, <b>무신사</b>는 피해자인가?', '<b>무신사</b> 에센셜 <b>가품</b> 판매 <b>논란</b>, 크림과 경쟁구도에서 승리할 수... ', '<b>무신사</b> vs 크림 <b>가품</b>결과 및 입막음 <b>논란</b>', '<b>무신사</b> 크림 <b>가품논란</b> 최종 결말', '<b>무신사 가품논란</b>, 200% 환불', '에센셜 <b>가품 논란</b> &lt;크림&gt;승리, 체면 구긴 &lt;<b>무신사</b>&gt;', '크림 <b>무신사 가품</b> 싸움, 피어오브갓에도 불똥이 튀었다?', '<b>무신사</b>와 네이버 크림의 에센셜 티셔츠 정,<b>가품 논란</b>/ 누구... ', '<b>무신사</b> 솔드아웃 <b>가품 논란</b>, 늦장 검수대기, 검수기간까지... ', '크림 <b>무신사</b> 피어오브갓 정,<b>가품 논란</b>! 영혼의 맞다이! 멸망전... ', '티몬도 새벽배송 한다!+<b>가품논란</b>에 고개 숙인 <b>무신사</b>', '[<b>무신사</b>vs크림] <b>가품논란</b> 최종정리 :: 멸망전 법정싸움 시작', "티셔츠' <b>가품 논란</b>...<b>가품</b> 맞다 결론! <b>무신사</b> &quot;200% 보상&quot;", '<b>무신사</b> VS 크림 피오오브갓... 티셔츠 <b>가품 논란</b> 총 정리... ', '<b>가품 논란</b>이 생겨도 <b

C:\Users\User\AppData\Local\Temp/ipykernel_580/1653698608.py:66: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(os.path.abspath('chromedriver')) #또는 driver.exe 가있는 파일 위치 복사하여 입력
C:\Users\User\AppData\Local\Temp/ipykernel_580/1653698608.py:79: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  a = driver.find_element_by_css_selector('div.se-main-container').text
C:\Users\User\AppData\Local\Temp/ipykernel_580/1653698608.py:83: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  a = driver.find_element_by_css_selector('div#content-area').text


<<본문 크롤링이 완료되었습니다.>>


,제목,링크,내용
0,"무신사 크림 가품 논란, 에센셜 티셔츠 정품 인증에 대한 짝퉁...",https://blog.naver.com/shiaru/222659267209,이 글에 사용된 이미지는 유료 라이선스 구매후 사용중입니다.\n안녕하세요. 오늘은 ...
1,무신사 크림 피어오브갓 에센셜티셔츠 가품 논란 결과는?,https://blog.naver.com/yonfun/222696173380,#무신사크림가품논란 #무신사크림논란 #무신사크림가품 #에센셜티셔츠논란\n국내 패션플...
2,무신사 - 크림 피어오브갓 에센셜 티셔츠 가품 논란,https://blog.naver.com/leegeehee311/222689117171,올초부터 핫한 이슈였던 \n무신사와 크림의 피어오브갓 에센셜 티셔츠 가품 전쟁이 오...
3,"골프의 계절, MZ세대를 노려라!+무신사VS크림, 가품논란...",https://blog.naver.com/d_idea/222657566279,"골프의 계절, MZ세대를 노려라!\n겨울이 끝나고 봄이 오면 야외활동이 늘어나게 되..."
4,"[유통이슈] 무신사, 크림 가품 논란에 대해 알아봅시다!",https://blog.naver.com/onchannelofficial/22270...,안녕하세요! 온채널 입니다!\n오늘 알아볼 정보는 '무신사'(패션 쇼핑 플랫폼)와 ...
...,...,...,...
95,[News] 무신사는 어쩌다 '짭신사'가 됐나,https://blog.naver.com/levvit_space/222694045778,http://news.bizwatch.co.kr/article/consumer/20...
96,"무신사 vs 크림, 지는 쪽 끝장....? 누가 이길까",https://blog.naver.com/vldzmqkffkf/222659620407,3자에게만 재밌는 이슈가 생겼죠.\n바로 무신사 가품 논란입니다.\n(늘 그렇지만 ...
97,"무신사 VS 크림, 피어오브갓 에센셜 직접 등판! 승자는 크림?!",https://blog.naver.com/ektmf8910/222689172959,"한달 전, 무신사와 크림이 피어오브갓 에션셜 아플리케 티셔츠 가품 논란으로 떠들썩했..."
98,"무신사와 크림의 ‘가품공방’, 그 결과는?",https://blog.naver.com/moningii/222693117407,공지 2018바이오혁신성장대전 2018바이오미래포럼 모닝엔터컴\n2018. 11. ...


In [6]:
df.to_csv('df_1.csv', index=False, header=False, encoding='utf-8-sig') # encoding='utf-8-sig' : 크롤링 후 csv에 저장할 때

In [7]:
import psycopg2

In [8]:
conn_str = 'host=localhost dbname=postgres user=postgres password=1234 port=5432'
try: 
    conn = psycopg2.connect(conn_str)
    print('=====접속 성공=====')
except psycopg2.DatabaseError as db_err:
    print('접속오류!!')
    print(db_err)

=====접속 성공=====


In [ ]:
# 커서 생성
cur = conn.cursor()
# 실행할 때마다 다른 값이 나오지 않게 테이블 제거
cur.execute('DROP TABLE IF EXISTS naver_blog1')

try:
    for i in df.iterrows:
        cur.execute('INSERT INTO naver_blog1 (Title, Originallink, Pubdate) VALUES (%s, %s, %d);', (i) ) # sql 문장 
    
    # 커밋하기
    conn.commit() 
    print("----- insert ok") 
except: 
    conn.rollback() 
    print("--- DB error")